# 根因分析

In [34]:
import os
import pandas as pd
import numpy as np
import time
from sklearn import preprocessing
from collections import Counter
from tqdm import *


train= pd.read_csv('训练告警.csv',encoding='gbk', engine='python')
test= pd.read_csv('测试告警.csv',encoding='gbk', engine='python')
train.columns=['告警开始时间', '告警标题', '涉及基站eNBID或小区ECGI']
test.columns=['告警开始时间', '告警标题', '涉及基站eNBID或小区ECGI']

train_label= pd.read_csv('训练故障工单.csv', encoding="gbk", engine='python')
test_label= pd.read_csv('测试故障工单.csv',encoding="gbk", engine='python')

#去掉空行
train=train[~train['涉及基站eNBID或小区ECGI'].isnull()].reset_index(drop=True)
train=train.drop_duplicates(['告警开始时间', '告警标题', '涉及基站eNBID或小区ECGI']).reset_index(drop=True)
train_label=train_label.drop_duplicates(['故障发生时间', '故障原因定位（大类）', '涉及基站eNBID或小区ECGI']).reset_index(drop=True)
test=test.drop_duplicates(['告警开始时间', '告警标题', '涉及基站eNBID或小区ECGI']).reset_index(drop=True)

train['告警开始时间']=train['告警开始时间'].apply(lambda x:time.strptime(x,"%Y/%m/%d %H:%M"))
test['告警开始时间']=test['告警开始时间'].apply(lambda x:time.strptime(x,"%Y/%m/%d %H:%M"))

train_label['故障发生时间']=train_label['故障发生时间'].apply(lambda x:time.strptime(x,"%Y/%m/%d %H:%M"))
test_label['故障发生时间']=test_label['故障发生时间'].apply(lambda x:time.strptime(x,"%Y/%m/%d %H:%M"))

train['告警开始时间_timestamp']=train['告警开始时间'].apply(lambda x:int(time.mktime(x)))
test['告警开始时间_timestamp']=test['告警开始时间'].apply(lambda x:int(time.mktime(x)))

train_label['故障发生时间_timestamp']=train_label['故障发生时间'].apply(lambda x:int(time.mktime(x)))
test_label['故障发生时间_timestamp']=test_label['故障发生时间'].apply(lambda x:int(time.mktime(x)))

# 衍生告警是依赖于根源告警的，这里我们需要删除掉
train=train[ ~ train['告警标题'].str.contains('衍生告警')]
test=test[ ~ test['告警标题'].str.contains('衍生告警')] 

In [35]:
# 整合训练集和测试集
total_data = pd.concat([train,test],axis=0).reset_index(drop=True)
total_data = total_data.drop_duplicates().reset_index(drop=True)
total_data

,告警开始时间,告警标题,涉及基站eNBID或小区ECGI,告警开始时间_timestamp
0,"(2019, 2, 5, 0, 1, 0, 1, 36, -1)",射频单元IR接口异常告警,383A218145EEA7892DA45BC4B2DFCBFB,1549296060
1,"(2019, 2, 5, 0, 1, 0, 1, 36, -1)",射频单元IR接口异常告警,2216E5B350E675E20B983E9D75F6B232,1549296060
2,"(2019, 2, 5, 0, 2, 0, 1, 36, -1)",射频单元交流掉电告警,3C8F2AA2E939E9608CA5FE2200E87D84,1549296120
3,"(2019, 2, 5, 0, 2, 0, 1, 36, -1)",射频单元业务不可用告警,3C8F2AA2E939E9608CA5FE2200E87D84,1549296120
4,"(2019, 2, 5, 0, 2, 0, 1, 36, -1)",射频单元光模块收发异常告警,713570937709ED63AFD495B85CD3B5B3,1549296120
5,"(2019, 2, 5, 0, 3, 0, 1, 36, -1)",射频单元维护链路异常告警,3C8F2AA2E939E9608CA5FE2200E87D84,1549296180
6,"(2019, 2, 5, 0, 3, 0, 1, 36, -1)",系统时钟不可用告警,A44F8E5DAC2C7FBA9D21DCBACD123D26,1549296180
7,"(2019, 2, 5, 0, 4, 0, 1, 36, -1)",射频单元ALD电流异常告警,679044BD425203C6624350E6645C8BAB,1549296240
8,"(2019, 2, 5, 0, 4, 0, 1, 36, -1)",射频单元驻波告警,0E71C7CD187B81097390638A760D5CE9,1549296240
9,"(2019, 2, 5, 0, 4, 0, 1, 36, -1)",系统时钟不可用告警,A03C287D59BE41CC3407A05D6029EF7F,1549296240


In [36]:
train.nunique()

告警开始时间              76808
告警标题                   76
涉及基站eNBID或小区ECGI     9033
告警开始时间_timestamp    76808
dtype: int64

In [37]:
total_data.nunique()

告警开始时间              77014
告警标题                   77
涉及基站eNBID或小区ECGI    10218
告警开始时间_timestamp    77014
dtype: int64

In [49]:
netid_in_total = total_data['涉及基站eNBID或小区ECGI'].unique()
title_in_total = total_data['告警标题'].unique()
netid_in_train = train['涉及基站eNBID或小区ECGI'].unique()
title_in_train = train['告警标题'].unique()
title_in_train
# 置信度比较低的预测
# not_confident_predict = ((test['涉及基站eNBID或小区ECGI'].isin(netid_in_train) == False) & (test_data['AREA'].isin(area_in_train) == False))
not_confident_predict1 = test['涉及基站eNBID或小区ECGI'].isin(netid_in_train) == False
not_confident_predict2 = test['告警标题'].isin(title_in_train) == False
test[not_confident_predict1], test[not_confident_predict2]

(                                     告警开始时间               告警标题  \
 0          (2019, 2, 5, 0, 1, 0, 1, 36, -1)  小区接收通道干扰噪声功率不平衡告警   
 6          (2019, 2, 5, 0, 6, 0, 1, 36, -1)       BBU IR接口异常告警   
 9          (2019, 2, 5, 0, 9, 0, 1, 36, -1)       射频单元IR接口异常告警   
 15        (2019, 2, 5, 0, 12, 0, 1, 36, -1)         射频单元交流掉电告警   
 16        (2019, 2, 5, 0, 13, 0, 1, 36, -1)       射频单元维护链路异常告警   
 ...                                     ...                ...   
 274232  (2019, 3, 31, 22, 56, 0, 6, 90, -1)       射频单元维护链路异常告警   
 274236  (2019, 3, 31, 22, 57, 0, 6, 90, -1)           4G基站退服告警   
 274303  (2019, 3, 31, 23, 18, 0, 6, 90, -1)       射频单元IR接口异常告警   
 274356  (2019, 3, 31, 23, 42, 0, 6, 90, -1)           星卡天线故障告警   
 274377  (2019, 3, 31, 23, 52, 0, 6, 90, -1)           星卡天线故障告警   
 
                         涉及基站eNBID或小区ECGI  告警开始时间_timestamp  
 0       2020EBF80F30C27D6FB98ADAF98D04C7        1549296060  
 6       6A231A366DF4C8025EC71DC0D22A5327        1549296360  
 9      

In [78]:
test_label [test_label['涉及基站eNBID或小区ECGI'].isin(netid_in_total) == False] # 训练集中的网元不在总的信息中, 这时候应该选择跟他临近的值
# 经分析 我们选择 0.700 电力故障 0.300 硬件故障
# train_label[train_label['故障发生时间_timestamp'].isin(range(1553024700, 1553024800))]

,工单编号,故障发生时间,涉及基站eNBID或小区ECGI,故障原因定位（大类）,故障发生时间_timestamp
3479,3480,"(2019, 3, 20, 3, 46, 0, 2, 79, -1)",EAE7032677038905849E6140D1206A97,NaN,1553024760


In [76]:
test_label[test_label['涉及基站eNBID或小区ECGI'].isin(netid_in_train) == False].nunique()

工单编号                1269
故障发生时间              1193
涉及基站eNBID或小区ECGI    1186
故障原因定位（大类）             0
故障发生时间_timestamp    1193
dtype: int64

### 从上面可知有1000多个网元不在这个测试集，1个未知的标题， 我们需要通过关联分析挖掘其中的规律

In [45]:
train_label.head()

,工单编号,故障发生时间,涉及基站eNBID或小区ECGI,故障原因定位（大类）,故障发生时间_timestamp
0,1,"(2019, 2, 4, 22, 26, 0, 0, 35, -1)",5F0A37BA3C8428C1B2ED83CEB714BA3F,电力故障,1549290360
1,2,"(2019, 2, 5, 0, 12, 0, 1, 36, -1)",505346F7A6D536667EE8F73CD15C1179,电力故障,1549296720
2,3,"(2019, 2, 5, 0, 32, 0, 1, 36, -1)",597ACECFDC702C079940F608FA17BE76,电力故障,1549297920
3,4,"(2019, 2, 5, 0, 34, 0, 1, 36, -1)",F98EC4E70CBBD66A4005CC3EF9D8233D,电力故障,1549298040
4,5,"(2019, 2, 5, 0, 36, 0, 1, 36, -1)",8B52CD71E5397D8D818F641A56657863,电力故障,1549298160


In [47]:
train_label [train_label['涉及基站eNBID或小区ECGI'].isin(netid_in_total) == False] # 训练集中的网元不在总的信息中， 这里训练应该丢掉

,工单编号,故障发生时间,涉及基站eNBID或小区ECGI,故障原因定位（大类）,故障发生时间_timestamp
12857,13742,"(2019, 3, 20, 4, 1, 0, 2, 79, -1)",352615346B00C109AE6BCFFD1A84DF1F,电力故障,1553025660
12886,13772,"(2019, 3, 20, 4, 57, 0, 2, 79, -1)",B89FED2E1608D840FF3910360C145B0C,硬件故障,1553029020
12971,13860,"(2019, 3, 20, 6, 30, 0, 2, 79, -1)",7DB909C35B86D5318AE088818FA92CEF,电力故障,1553034600
12989,13879,"(2019, 3, 20, 6, 49, 0, 2, 79, -1)",DF834C946DFC5B42D48219604DD1624A,电力故障,1553035740


In [79]:
# train[train['告警开始时间_timestamp'].isin(range(1553024760, 1553024761))]

In [80]:
test_label [test_label['涉及基站eNBID或小区ECGI'].isin(netid_in_total) == False] # 训练集中的网元不在总的信息中

,工单编号,故障发生时间,涉及基站eNBID或小区ECGI,故障原因定位（大类）,故障发生时间_timestamp
3479,3480,"(2019, 3, 20, 3, 46, 0, 2, 79, -1)",EAE7032677038905849E6140D1206A97,NaN,1553024760


In [83]:
train_label["故障原因定位（大类）"].value_counts()

电力故障    14277
硬件故障     2053
传输故障     1861
软件故障      225
动环故障      110
误告警        14
Name: 故障原因定位（大类）, dtype: int64

## 使用时间滑窗创建时间序列



In [108]:
save_list = []
new_data=pd.DataFrame(columns=["涉及基站eNBID或小区ECGI", "告警开始时间_timestamp", "故障发生时间_timestamp", "故障原因定位（大类）"])
for i,row in train_label.iterrows():
    m = total_data[(total_data['涉及基站eNBID或小区ECGI']==row['涉及基站eNBID或小区ECGI'])& \
            (total_data['告警开始时间_timestamp'] >= row['故障发生时间_timestamp']) & \
           (total_data['告警开始时间_timestamp'] <= row['故障发生时间_timestamp']+10)].reset_index(drop=True)
    if m.shape[0] == 0:
        continue
    m["故障原因定位（大类）"] =  row['故障原因定位（大类）']
    m["故障发生时间_timestamp"] = row["故障发生时间_timestamp"]
    save_list.append(set(m["告警标题"]) | set([row['涉及基站eNBID或小区ECGI'], row['故障原因定位（大类）']]) )
    new_data = new_data.append(m)
new_data

,告警开始时间,告警开始时间_timestamp,告警标题,故障原因定位（大类）,故障发生时间_timestamp,涉及基站eNBID或小区ECGI
0,"(2019, 2, 5, 0, 12, 0, 1, 36, -1)",1549296720,小区不可用告警,电力故障,1549296720,505346F7A6D536667EE8F73CD15C1179
0,"(2019, 2, 5, 0, 32, 0, 1, 36, -1)",1549297920,小区接收通道干扰噪声功率不平衡告警,电力故障,1549297920,597ACECFDC702C079940F608FA17BE76
0,"(2019, 2, 5, 0, 34, 0, 1, 36, -1)",1549298040,小区接收通道干扰噪声功率不平衡告警,电力故障,1549298040,F98EC4E70CBBD66A4005CC3EF9D8233D
0,"(2019, 2, 5, 0, 36, 0, 1, 36, -1)",1549298160,4G基站退服告警,电力故障,1549298160,8B52CD71E5397D8D818F641A56657863
0,"(2019, 2, 5, 0, 41, 0, 1, 36, -1)",1549298460,时钟参考源异常告警,硬件故障,1549298460,D28C3879A688B51BC7016E55131A3F2D
0,"(2019, 2, 5, 1, 24, 0, 1, 36, -1)",1549301040,小区不可用告警,硬件故障,1549301040,67E65EC1EE29F3971993424FD9DCEAA1
0,"(2019, 2, 5, 2, 9, 0, 1, 36, -1)",1549303740,射频单元驻波告警,传输故障,1549303740,476A1EBA3437F8117AD6C5D40C551FF7
0,"(2019, 2, 5, 2, 29, 0, 1, 36, -1)",1549304940,射频单元CPRI接口异常告警,电力故障,1549304940,B731EF07B1376AEE94C525C0ED8AEE88
0,"(2019, 2, 5, 2, 32, 0, 1, 36, -1)",1549305120,网元连接中断,硬件故障,1549305120,0C965544227EF071A4D6C247F77D0617
0,"(2019, 2, 5, 2, 40, 0, 1, 36, -1)",1549305600,网元连接中断,电力故障,1549305600,DB8AFB92757886A789D95AFB6D34300D


In [112]:
# 保存结果
new_data.to_csv('train_set.csv',index=None,encoding='utf8')
with open("seq.csv", "w", encoding="utf8") as f:
    f.write("\n".join([",".join(x) for x in save_list]))